In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


import warnings
warnings.filterwarnings('ignore')

sns.set(style="whitegrid")

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
df = pd.read_csv('datasets/global_master_dataset_fixed.csv')

## PREPROCESSING UTILITIES

In [11]:
def prepare_numerical(df, numerical_features):
    """Impute + scale numerical features."""
    imputer = SimpleImputer(strategy="median")
    scaler = StandardScaler()

    X_num = df[numerical_features].copy()
    X_imp = imputer.fit_transform(X_num)
    X_scaled = scaler.fit_transform(X_imp)
    
    return X_scaled, imputer, scaler


def prepare_categorical(df, categorical_features):
    """Label encode categorical variables."""
    encoders = {}
    X_cat_encoded = {}

    for col in categorical_features:
        le = LabelEncoder()
        X_cat_encoded[col] = le.fit_transform(df[col].fillna("Unknown"))
        encoders[col] = le

    return X_cat_encoded, encoders


## BUILD NEURAL NETWORK MODEL

In [12]:
def build_nn_model(n_num_features, encoders):
    """Create feed-forward NN with embeddings."""
    
    # Numerical
    num_in = layers.Input(shape=(n_num_features,), name="num_in")

    # Embedding dims
    emb_sizes = {
        "Country": min(50, (len(encoders["Country"].classes_) + 1) // 2),
        "Region":  min(20, (len(encoders["Region"].classes_) + 1) // 2),
        "Crop":    min(50, (len(encoders["Crop"].classes_) + 1) // 2)
    }

    # Inputs
    c_in = layers.Input(shape=(1,), name="country_in")
    r_in = layers.Input(shape=(1,), name="region_in")
    cr_in = layers.Input(shape=(1,), name="crop_in")

    # Embeddings
    c_emb = layers.Flatten()(layers.Embedding(len(encoders["Country"].classes_), emb_sizes["Country"])(c_in))
    r_emb = layers.Flatten()(layers.Embedding(len(encoders["Region"].classes_), emb_sizes["Region"])(r_in))
    cr_emb = layers.Flatten()(layers.Embedding(len(encoders["Crop"].classes_), emb_sizes["Crop"])(cr_in))

    # Combine
    x = layers.Concatenate()([num_in, c_emb, r_emb, cr_emb])

    # Dense layers
    x = layers.Dense(256, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(128, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.1)(x)

    out = layers.Dense(1)(x)

    model = keras.Model(inputs=[num_in, c_in, r_in, cr_in], outputs=out)
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse", metrics=["mae"])

    return model, emb_sizes


## TRAINING FUNCTION FOR ONE CATEGORY

In [14]:
def train_single_category(df_cat, numerical_features, categorical_features, category_name):

    print(f"\n===== Training Category: {category_name} =====")
    print(f"Samples: {len(df_cat)} | Unique Crops: {df_cat['Crop'].nunique()}")

    df_cat['Log_Yield'] = np.log1p(df_cat['Yield'])


    X_num_scaled, imputer, scaler = prepare_numerical(df_cat, numerical_features)
    X_cat_encoded, encoders = prepare_categorical(df_cat, categorical_features)
    y = df_cat['Log_Yield'].values


    X_num_train, X_num_test, y_train, y_test = train_test_split(
        X_num_scaled, y, test_size=0.2, random_state=42
    )

    X_cat_train = {}
    X_cat_test = {}
    for col in categorical_features:
        X_cat_train[col], X_cat_test[col] = train_test_split(
            X_cat_encoded[col], test_size=0.2, random_state=42
        )


    model, emb_dims = build_nn_model(X_num_train.shape[1], encoders)


    es = callbacks.EarlyStopping(monitor="val_loss", patience=15, restore_best_weights=True, verbose=0)
    rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5,
                                        patience=5, min_lr=1e-6, verbose=0)

 
    history = model.fit(
        [X_num_train, X_cat_train['Country'], X_cat_train['Region'], X_cat_train['Crop']],
        y_train,
        validation_split=0.15,
        epochs=100,
        batch_size=128,
        callbacks=[es, rlrop],
        verbose=0
    )

  
    y_pred_log = model.predict(
        [X_num_test, X_cat_test['Country'], X_cat_test['Region'], X_cat_test['Crop']],
        verbose=0
    ).flatten()

    y_pred = np.expm1(y_pred_log)
    y_test_orig = np.expm1(y_test)

  
    r2 = r2_score(y_test_orig, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
    mae = mean_absolute_error(y_test_orig, y_pred)

    print(f"R²: {r2:.4f} | RMSE: {rmse:.0f} | MAE: {mae:.0f}")


    model.save(f"models/nn_{category_name.replace(' ','_').lower()}.keras")

    return {
        "Category": category_name,
        "R2": r2,
        "RMSE": rmse,
        "MAE": mae,
        "Epochs": len(history.history['loss']),
        "y_test": y_test_orig,
        "y_pred": y_pred
    }


## LOOP OVER ALL CATEGORIES + STORE RESULTS

In [15]:
all_results = []
category_predictions = {}

for category in valid_categories:
    df_cat = df_filtered[df_filtered['Crop_Category'] == category].copy()

    result = train_single_category(
        df_cat=df_cat,
        numerical_features=numerical_features,
        categorical_features=['Country', 'Region', 'Crop'],
        category_name=category
    )

    all_results.append(result)
    category_predictions[category] = {
        "y_test": result["y_test"],
        "NN": result["y_pred"]
    }

pd.DataFrame(all_results)



===== Training Category: Other =====
Samples: 50475 | Unique Crops: 69
R²: 0.8701 | RMSE: 3795 | MAE: 1536

===== Training Category: Vegetables =====
Samples: 18028 | Unique Crops: 20
R²: 0.8388 | RMSE: 5773 | MAE: 3406

===== Training Category: Fruits =====
Samples: 16978 | Unique Crops: 24
R²: 0.8340 | RMSE: 5011 | MAE: 2829

===== Training Category: Cereals =====
Samples: 12109 | Unique Crops: 17
R²: 0.7407 | RMSE: 2036 | MAE: 772

===== Training Category: Legumes =====
Samples: 11880 | Unique Crops: 19
R²: 0.8474 | RMSE: 1507 | MAE: 593

===== Training Category: Industrial Crops =====
Samples: 5971 | Unique Crops: 14
R²: 0.9454 | RMSE: 4596 | MAE: 1899

===== Training Category: Root Crops =====
Samples: 4748 | Unique Crops: 6
R²: 0.6346 | RMSE: 6298 | MAE: 4203

===== Training Category: Oil Crops =====
Samples: 4301 | Unique Crops: 8
R²: 0.8235 | RMSE: 1962 | MAE: 862


,Category,R2,RMSE,MAE,Epochs,y_test,y_pred
0,Other,0.870069,3794.767993,1535.968927,100,"[2406.0, 4061.299999999999, 852.0000000000001,...","[2680.091, 5075.8384, 1324.1494, 942.2334, 446..."
1,Vegetables,0.838790,5773.243696,3406.398828,100,"[43280.19999999998, 3222.2000000000016, 16333....","[33714.293, 3777.9004, 16469.102, 7431.348, 95..."
2,Fruits,0.834017,5011.074333,2828.760237,100,"[42023.499999999985, 18319.500000000004, 2904....","[36510.918, 26629.023, 3472.5889, 20678.816, 4..."
3,Cereals,0.740742,2035.859361,772.393552,100,"[2733.2999999999993, 28225.000000000007, 999.9...","[1299.0883, 19376.486, 1491.6418, 1062.6964, 8..."
4,Legumes,0.847399,1507.254354,593.301395,100,"[1134.1000000000001, 999.9999999999999, 4956.2...","[1083.0709, 1657.0963, 2194.136, 688.318, 2312..."
5,Industrial Crops,0.945369,4595.884920,1899.090315,100,"[912.5000000000003, 61625.500000000015, 872.30...","[333.64087, 63713.18, 764.7573, 1980.7354, 162..."
6,Root Crops,0.634566,6297.919181,4202.543771,97,"[1864.4, 9430.8, 15767.800000000003, 1822.2000...","[1767.1613, 11688.235, 8475.367, 4255.222, 111..."
7,Oil Crops,0.823514,1962.385362,862.060803,100,"[686.7000000000002, 7716.199999999995, 1033.09...","[717.1249, 7463.925, 1359.469, 2084.4746, 4172..."


In [16]:
results_df = pd.DataFrame(all_results)
print("\n" + results_df.to_string(index=False))

category_sizes = {cat: len(df_filtered[df_filtered['Crop_Category'] == cat]) for cat in valid_categories}
total_samples = sum(category_sizes.values())

weighted_r2 = sum(row['R2'] * category_sizes[row['Category']] for _, row in results_df.iterrows()) / total_samples
weighted_rmse = sum(row['RMSE'] * category_sizes[row['Category']] for _, row in results_df.iterrows()) / total_samples
weighted_mae = sum(row['MAE'] * category_sizes[row['Category']] for _, row in results_df.iterrows()) / total_samples



        Category       R2        RMSE         MAE  Epochs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [22]:
print(f"   R²:   {weighted_r2:.4f}")
print(f"   RMSE: {weighted_rmse:,.2f} kg/ha")
print(f"   MAE:  {weighted_mae:,.2f} kg/ha")

   R²:   0.8389
   RMSE: 3,928.36 kg/ha
   MAE:  1,914.75 kg/ha


## Hyperparameter Tuning.

In [19]:
import keras_tuner as kt

def build_tunable_model(hp, n_num_features, encoders):

    # EMBEDDING DIMENSIONS
    emb_country_dim = hp.Int("emb_country_dim", min_value=16, max_value=64, step=8)
    emb_region_dim  = hp.Int("emb_region_dim", min_value=8, max_value=32, step=4)
    emb_crop_dim    = hp.Int("emb_crop_dim", min_value=16, max_value=64, step=8)

    # Inputs
    num_in = layers.Input(shape=(n_num_features,), name="num_in")
    c_in   = layers.Input(shape=(1,), name="country_in")
    r_in   = layers.Input(shape=(1,), name="region_in")
    cr_in  = layers.Input(shape=(1,), name="crop_in")

    # Embeddings
    c_emb = layers.Flatten()(layers.Embedding(len(encoders["Country"].classes_), emb_country_dim)(c_in))
    r_emb = layers.Flatten()(layers.Embedding(len(encoders["Region"].classes_),  emb_region_dim)(r_in))
    cr_emb = layers.Flatten()(layers.Embedding(len(encoders["Crop"].classes_),    emb_crop_dim)(cr_in))

    # Concatenate
    x = layers.Concatenate()([num_in, c_emb, r_emb, cr_emb])

    # Dense hidden layers
    for i in range(hp.Int("num_layers", 2, 5)):
        x = layers.Dense(
            units=hp.Int(f"units_{i}", min_value=64, max_value=512, step=64),
            activation="relu"
        )(x)
        x = layers.Dropout(hp.Float(f"dropout_{i}", 0.1, 0.5, step=0.1))(x)

    # Output
    out = layers.Dense(1)(x)

    # Learning rate search
    lr = hp.Float("lr", 1e-4, 5e-3, sampling="log")

    model = keras.Model(inputs=[num_in, c_in, r_in, cr_in], outputs=out)
    model.compile(optimizer=keras.optimizers.Adam(lr), loss="mse", metrics=["mae"])

    return model


### TUNING FUNCTION FOR ONE CATEGORY

In [20]:
def tune_model(df_cat, numerical_features, categorical_features, category_name):

    print(f"\n--- Hyperparameter Tuning: {category_name} ---")

    df_cat['Log_Yield'] = np.log1p(df_cat['Yield'])

    X_num_scaled, imputer, scaler = prepare_numerical(df_cat, numerical_features)
    X_cat_encoded, encoders = prepare_categorical(df_cat, categorical_features)
    y = df_cat['Log_Yield'].values

    X_num_train, X_num_test, y_train, y_test = train_test_split(
        X_num_scaled, y, test_size=0.2, random_state=42
    )

    X_cat_train = {col: None for col in categorical_features}
    X_cat_test  = {col: None for col in categorical_features}

    for col in categorical_features:
        X_cat_train[col], X_cat_test[col] = train_test_split(
            X_cat_encoded[col], test_size=0.2, random_state=42
        )

    # ---- BAYESIAN TUNER ----
    tuner = kt.BayesianOptimization(
        lambda hp: build_tunable_model(hp, X_num_train.shape[1], encoders),
        objective="val_loss",
        max_trials=20,
        directory="tuning",
        project_name=f"{category_name.replace(' ', '_')}_tuner"
    )

    tuner.search(
        [X_num_train, X_cat_train['Country'], X_cat_train['Region'], X_cat_train['Crop']],
        y_train,
        validation_split=0.15,
        epochs=50,
        batch_size=128,
        callbacks=[callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
        verbose=1
    )

    best_hp = tuner.get_best_hyperparameters(1)[0]
    model = tuner.hypermodel.build(best_hp)

    history = model.fit(
        [X_num_train, X_cat_train['Country'], X_cat_train['Region'], X_cat_train['Crop']],
        y_train,
        validation_split=0.2,
        epochs=80,
        batch_size=128,
        verbose=0
    )

    y_pred_log = model.predict(
        [X_num_test, X_cat_test['Country'], X_cat_test['Region'], X_cat_test['Crop']]
    ).flatten()

    y_pred = np.expm1(y_pred_log)
    y_test_orig = np.expm1(y_test)

    r2 = r2_score(y_test_orig, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test_orig, y_pred))
    mae = mean_absolute_error(y_test_orig, y_pred)

    print(f"TUNED RESULTS → R²={r2:.4f}, RMSE={rmse:.0f}, MAE={mae:.0f}")

    return model, best_hp, r2, rmse, mae


### RUN TUNING FOR ALL CATEGORIES

In [21]:
tuning_results = []

for category in valid_categories:
    df_cat = df_filtered[df_filtered["Crop_Category"] == category].copy()

    model, best_hp, r2, rmse, mae = tune_model(
        df_cat,
        numerical_features,
        ["Country", "Region", "Crop"],
        category
    )

    tuning_results.append({
        "Category": category,
        "R2": r2,
        "RMSE": rmse,
        "MAE": mae,
        "Best HP": best_hp.values
    })

pd.DataFrame(tuning_results)


Trial 20 Complete [00h 00m 07s]
val_loss: 0.2598671615123749

Best val_loss So Far: 0.10269518941640854
Total elapsed time: 00h 01m 35s
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
TUNED RESULTS → R²=0.8025, RMSE=2076, MAE=997


,Category,R2,RMSE,MAE,Best HP
0,Other,0.897179,3375.746470,1346.191640,"{'emb_country_dim': 56, 'emb_region_dim': 8, '..."
1,Vegetables,0.787998,6620.543442,3761.933673,"{'emb_country_dim': 64, 'emb_region_dim': 24, ..."
2,Fruits,0.760007,6025.565605,3158.458693,"{'emb_country_dim': 64, 'emb_region_dim': 24, ..."
3,Cereals,0.809179,1746.605802,591.089091,"{'emb_country_dim': 56, 'emb_region_dim': 8, '..."
4,Legumes,0.895692,1246.141562,512.085224,"{'emb_country_dim': 48, 'emb_region_dim': 20, ..."
5,Industrial Crops,0.919207,5588.997850,2076.469994,"{'emb_country_dim': 64, 'emb_region_dim': 8, '..."
6,Root Crops,0.649626,6166.784380,4090.600724,"{'emb_country_dim': 64, 'emb_region_dim': 28, ..."
7,Oil Crops,0.802516,2075.846814,996.947917,"{'emb_country_dim': 56, 'emb_region_dim': 8, '..."
